## This notebook will be mainly used for the Data Science capstone project offered by IBM. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [4]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [5]:
#Lets have a look at the data
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [6]:
# Since all the relevant data is in the features key
neighborhoods_data = newyork_data['features']

In [7]:
#Let's take a look at the first item in this list.
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [8]:
#Transforming this data of nested Python dictionaries into a pandas dataframe.

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [9]:
#Looping through the data and filling the dataframe one row at a time.
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [10]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [11]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [12]:
#Unique boroughs
neighborhoods.Borough.unique()

array(['Bronx', 'Manhattan', 'Brooklyn', 'Queens', 'Staten Island'],
      dtype=object)

In [13]:
#Use geopy library to get the latitude and longitude values of New York City.
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [14]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [17]:
#Since we are focusing only on Staten Island borough
si_data = neighborhoods[neighborhoods['Borough'] == 'Staten Island'].reset_index(drop=True)
print('The borough neighborhood table size is: {}'.format(si_data.shape))
si_data.head()

The borough neighborhood table size is: (63, 4)


,Borough,Neighborhood,Latitude,Longitude
0,Staten Island,St. George,40.644982,-74.079353
1,Staten Island,New Brighton,40.640615,-74.087017
2,Staten Island,Stapleton,40.626928,-74.077902
3,Staten Island,Rosebank,40.615305,-74.069805
4,Staten Island,West Brighton,40.631879,-74.107182


In [18]:
#Getting geopraphical coordinates of Staten Island
address = 'Staten Island, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Staten Island are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Staten Island are 40.5834557, -74.1496048.


In [20]:
# create map of Staten Island using latitude and longitude values
map_si = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(si_data['Latitude'], si_data['Longitude'], si_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_si)  
    
map_si

In [1]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'sensored' # your Foursquare ID
CLIENT_SECRET = 'sensored' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
print("Credentials loaded!")

Credentials loaded!


In [23]:
#Exploring the first neighborhood
si_data.loc[0, 'Neighborhood']

'St. George'

In [24]:
#Getting its latitude and longitude
neighborhood_latitude = si_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = si_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = si_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of St. George are 40.6449815710044, -74.07935312512797.


In [25]:
#Getting the top 100 venues that are in St. George within a radius of 500 meters.
url='https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, 500, 100)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e8c7fb6edbcad001be8763a'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'totalResults': 34,
  'suggestedBounds': {'ne': {'lat': 40.6494815755044,
    'lng': -74.07343346476772},
   'sw': {'lat': 40.6404815665044, 'lng': -74.08527278548821}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6924c46d4a5938e7ac6c0',
       'name': 'A&S Pizzeria',
       'location': {'address': '87 Stuyvesant Pl',
        'crossStreet': 'Wall st',
        'lat': 40.64393953223924,
        'lng': -74.0776259226109,
 

In [26]:
#Function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [27]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,A&S Pizzeria,Pizza Place,40.643940,-74.077626
1,Beso,Tapas Restaurant,40.643306,-74.076508
2,Richmond County Bank Ballpark,Baseball Stadium,40.645056,-74.076864
3,Staten Island September 11 Memorial,Monument / Landmark,40.646767,-74.076510
4,Nike Factory Store,Sporting Goods Shop,40.645753,-74.077702


In [28]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

34 venues were returned by Foursquare.


In [29]:
#Function to repeat the same process to all the neighborhoods in Staten Island
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
#Running the above function on each neighborhood and create a new dataframe called si_venues.
si_venues = getNearbyVenues(names=si_data['Neighborhood'],
                                   latitudes=si_data['Latitude'],
                                   longitudes=si_data['Longitude']
                                  )

St. George
New Brighton
Stapleton
Rosebank
West Brighton
Grymes Hill
Todt Hill
South Beach
Port Richmond
Mariner's Harbor
Port Ivory
Castleton Corners
New Springville
Travis
New Dorp
Oakwood
Great Kills
Eltingville
Annadale
Woodrow
Tottenville
Tompkinsville
Silver Lake
Sunnyside
Park Hill
Westerleigh
Graniteville
Arlington
Arrochar
Grasmere
Old Town
Dongan Hills
Midland Beach
Grant City
New Dorp Beach
Bay Terrace
Huguenot
Pleasant Plains
Butler Manor
Charleston
Rossville
Arden Heights
Greenridge
Heartland Village
Chelsea
Bloomfield
Bulls Head
Richmond Town
Shore Acres
Clifton
Concord
Emerson Hill
Randall Manor
Howland Hook
Elm Park
Manor Heights
Willowbrook
Sandy Ground
Egbertville
Prince's Bay
Lighthouse Hill
Richmond Valley
Fox Hills


In [34]:
#Checking if the above code ran on each neighborhood or not
si_data['Neighborhood']

0            St. George
1          New Brighton
2             Stapleton
3              Rosebank
4         West Brighton
5           Grymes Hill
6             Todt Hill
7           South Beach
8         Port Richmond
9      Mariner's Harbor
10           Port Ivory
11    Castleton Corners
12      New Springville
13               Travis
14             New Dorp
15              Oakwood
16          Great Kills
17          Eltingville
18             Annadale
19              Woodrow
20          Tottenville
21        Tompkinsville
22          Silver Lake
23            Sunnyside
24            Park Hill
25          Westerleigh
26         Graniteville
27            Arlington
28             Arrochar
29             Grasmere
30             Old Town
31         Dongan Hills
32        Midland Beach
33           Grant City
34       New Dorp Beach
35          Bay Terrace
36             Huguenot
37      Pleasant Plains
38         Butler Manor
39           Charleston
40            Rossville
41        Arden 

In [35]:
#Checking the size of the resulting dataframe
print(si_venues.shape)
si_venues.head()

(837, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,St. George,40.644982,-74.079353,A&S Pizzeria,40.643940,-74.077626,Pizza Place
1,St. George,40.644982,-74.079353,Beso,40.643306,-74.076508,Tapas Restaurant
2,St. George,40.644982,-74.079353,Richmond County Bank Ballpark,40.645056,-74.076864,Baseball Stadium
3,St. George,40.644982,-74.079353,Staten Island September 11 Memorial,40.646767,-74.076510,Monument / Landmark
4,St. George,40.644982,-74.079353,Nike Factory Store,40.645753,-74.077702,Sporting Goods Shop


In [36]:
si_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Annadale,12,12,12,12,12,12
Arden Heights,4,4,4,4,4,4
Arlington,4,4,4,4,4,4
Arrochar,18,18,18,18,18,18
Bay Terrace,10,10,10,10,10,10
Bloomfield,5,5,5,5,5,5
Bulls Head,45,45,45,45,45,45
Butler Manor,6,6,6,6,6,6
Castleton Corners,15,15,15,15,15,15


In [37]:
print('There are {} uniques categories.'.format(len(si_venues['Venue Category'].unique())))

There are 176 uniques categories.


In [38]:
si_venues['Venue Category'].unique()

array(['Pizza Place', 'Tapas Restaurant', 'Baseball Stadium',
       'Monument / Landmark', 'Sporting Goods Shop', 'Burger Joint',
       'Italian Restaurant', 'Clothing Store', 'Theater', 'Bar',
       'Toy / Game Store', 'American Restaurant', 'Farmers Market',
       'Donut Shop', 'Coffee Shop', 'Pharmacy', 'Steakhouse',
       'Outlet Mall', 'Plaza', 'Snack Place', 'Sandwich Place',
       'Harbor / Marina', 'Scenic Lookout', 'Park', 'Bakery',
       'Tourist Information Center', 'Discount Store', 'Bus Stop',
       'Bowling Alley', 'Daycare', 'Deli / Bodega', 'Playground',
       'Sri Lankan Restaurant', 'Beer Bar', 'Spanish Restaurant',
       'Asian Restaurant', 'Skate Park',
       'Residential Building (Apartment / Condo)', 'Mexican Restaurant',
       'Restaurant', 'New American Restaurant', 'Bank',
       'Fast Food Restaurant', 'Department Store', 'Miscellaneous Shop',
       'Diner', 'Antique Shop', 'Performing Arts Venue', 'Café',
       'Cosmetics Shop', 'Breakfast Spot'

In [50]:
#Categories which could effect the Corporation sales or to note are: Baseball Stadium, Sporting Goods Shop, Breakfast Spot, Athletics & Sports, Spa, Gym, Gym / Fitness Center, Sports Club, Sports Bar, Golf Course, Basketball Court, Tanning Salon, Skating Rink, Supplement Shop, Yoga Studio, Smoothie Shop, Dance Studio
cat_eff = ['Neighborhood', 'Baseball Stadium', 'Sporting Goods Shop', 'Breakfast Spot', 'Athletics & Sports', 'Spa', 'Gym', 'Gym / Fitness Center', 'Sports Club', 'Sports Bar', 'Golf Course', 'Basketball Court', 'Tanning Salon', 'Skating Rink', 'Supplement Shop', 'Yoga Studio', 'Smoothie Shop', 'Dance Studio']

In [51]:
#To analyse each Neighborhood
# one hot encoding
si_onehot = pd.get_dummies(si_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
si_onehot['Neighborhood'] = si_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [si_onehot.columns[-1]] + list(si_onehot.columns[:-1])
si_onehot = si_onehot[fixed_columns]

si_onehot.head()

,Neighborhood,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Big Box Store,Board Shop,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Campground,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Furniture / Home Store,Gas Station,Gastropub,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Lawyer,Liquor Store,Locksmith,Market,Martial Arts Dojo,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Optical Shop,Outdoor Sculpture,Outdoors & Recreation,Outlet Mall,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Pub,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Salon / Barbershop,Sandwich Place,Scenic Lookout,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toll Plaza,Tourist Information Center,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Yoga Studio
0,St. George,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,St. George,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,St. George,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,St. George,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,St. George,0,0,0,0,0,0,0,0,0

In [52]:
si_grouped = si_onehot.groupby('Neighborhood').mean().reset_index()
si_grouped

,Neighborhood,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Big Box Store,Board Shop,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Campground,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Furniture / Home Store,Gas Station,Gastropub,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Lawyer,Liquor Store,Locksmith,Market,Martial Arts Dojo,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Optical Shop,Outdoor Sculpture,Outdoors & Recreation,Outlet Mall,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Pub,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Salon / Barbershop,Sandwich Place,Scenic Lookout,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toll Plaza,Tourist Information Center,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Yoga Studio
0,Annadale,0.083333,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.083333,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.0,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.083333,0.00

In [53]:
#To only focus neighborhoods based on effective venue categories
si_useful = si_grouped[cat_eff]

In [54]:
si_useful

,Neighborhood,Baseball Stadium,Sporting Goods Shop,Breakfast Spot,Athletics & Sports,Spa,Gym,Gym / Fitness Center,Sports Club,Sports Bar,Golf Course,Basketball Court,Tanning Salon,Skating Rink,Supplement Shop,Yoga Studio,Smoothie Shop,Dance Studio
0,Annadale,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.083333
1,Arden Heights,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,Arlington,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Arrochar,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,Bay Terrace,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
5,Bloomfield,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
6,Bulls Head,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022222,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
7,Butler Manor,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
8,Castleton Corners,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,Charleston,0.000000,0.034483,0.000000,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.034483,0.000000,0.000000,0.000000


In [74]:
#CRITICAL STEP-Getting the Neighborhoods which are best in either any of the useful venue categories
for x in si_useful.columns:
    print(si_useful.loc[si_useful[x] == max(si_useful[x])]) 

   Neighborhood  Baseball Stadium  Sporting Goods Shop  Breakfast Spot  \
62      Woodrow               0.0                  0.0             0.0   

    Athletics & Sports  Spa  Gym  Gym / Fitness Center  Sports Club  \
62                 0.0  0.0  0.0                   0.0          0.0   

    Sports Bar  Golf Course  Basketball Court  Tanning Salon  Skating Rink  \
62         0.0          0.0               0.0            0.0           0.0   

    Supplement Shop  Yoga Studio  Smoothie Shop  Dance Studio  
62              0.0          0.0            0.0           0.0  
   Neighborhood  Baseball Stadium  Sporting Goods Shop  Breakfast Spot  \
52   St. George          0.029412             0.058824             0.0   

    Athletics & Sports  Spa  Gym  Gym / Fitness Center  Sports Club  \
52                 0.0  0.0  0.0                   0.0          0.0   

    Sports Bar  Golf Course  Basketball Court  Tanning Salon  Skating Rink  \
52         0.0          0.0               0.0        

In [80]:
top_neighborhoods = [['Baseball Stadium', 'St. George'],
                      ['Sporting Goods Shop', 'St. George'],
                      ['Breakfast Spot', 'Pleasant Plains, West Brighton'],
                      ['Athletics & Sports, Gym / Fitness Center', 'Park Hill'],
                      ['Spa', 'Lighthouse Hill, Richmond Town'],
                      ['Gym', 'Sunnyside'],
                      ['Sports Club', 'Travis'],
                      ['Sports Bar, Dance Studio', 'Annadale'],
                      ['Sports Bar, Skating Rink', 'New Dorp Beach'],
                      ['Golf Course', 'Silver Lake'],
                      ['Yoga Studio', 'Shore Acres'],
                      ['Smoothie Shop', 'Annadale']]

In [81]:
top = pd.DataFrame(top_neighborhoods)

In [88]:
top.columns=['Venue Category', 'Neighborhood']

In [ ]:
folium.Map(location=[latitude, longitude], zoom_start=11)

In [55]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [56]:
#To get the most common useful venue neighborhood based
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = si_useful['Neighborhood']

for ind in np.arange(si_useful.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(si_useful.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Annadale,Dance Studio,Sports Bar,Yoga Studio,Supplement Shop,Skating Rink
1,Arden Heights,Dance Studio,Sports Club,Sporting Goods Shop,Breakfast Spot,Athletics & Sports
2,Arlington,Dance Studio,Sports Club,Sporting Goods Shop,Breakfast Spot,Athletics & Sports
3,Arrochar,Athletics & Sports,Dance Studio,Sports Club,Sporting Goods Shop,Breakfast Spot
4,Bay Terrace,Dance Studio,Sports Club,Sporting Goods Shop,Breakfast Spot,Athletics & Sports


In [58]:
neighborhoods_venues_sorted.shape

(63, 6)

In [60]:
#Run k-means to cluster the neighborhood into 5 clusters as we need to set up 5 places for the Corporation.
# set number of clusters
kclusters = 5

si_clustering = si_useful.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(si_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:5] 

array([1, 1, 1, 1, 1])

In [61]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

si_merged = si_data

# merge si_grouped with si_data to add latitude/longitude for each neighborhood
si_merged = si_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

si_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Staten Island,St. George,40.644982,-74.079353,1,Sporting Goods Shop,Baseball Stadium,Sports Club,Breakfast Spot,Athletics & Sports
1,Staten Island,New Brighton,40.640615,-74.087017,1,Dance Studio,Sports Club,Sporting Goods Shop,Breakfast Spot,Athletics & Sports
2,Staten Island,Stapleton,40.626928,-74.077902,1,Breakfast Spot,Dance Studio,Sports Club,Sporting Goods Shop,Athletics & Sports
3,Staten Island,Rosebank,40.615305,-74.069805,1,Breakfast Spot,Dance Studio,Sports Club,Sporting Goods Shop,Athletics & Sports
4,Staten Island,West Brighton,40.631879,-74.107182,1,Breakfast Spot,Dance Studio,Sports Club,Sporting Goods Shop,Athletics & Sports


In [62]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(si_merged['Latitude'], si_merged['Longitude'], si_merged['Neighborhood'], si_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [63]:
#Examining cluster 1
si_merged.loc[si_merged['Cluster Labels'] == 0, si_merged.columns[[1] + list(range(5, si_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
23,Sunnyside,Spa,Gym,Dance Studio,Sports Club,Sporting Goods Shop
43,Heartland Village,Spa,Dance Studio,Sports Club,Sporting Goods Shop,Breakfast Spot
47,Richmond Town,Spa,Dance Studio,Sports Club,Sporting Goods Shop,Breakfast Spot
56,Willowbrook,Spa,Dance Studio,Sports Club,Sporting Goods Shop,Breakfast Spot
60,Lighthouse Hill,Spa,Dance Studio,Sports Club,Sporting Goods Shop,Breakfast Spot


In [64]:
#Examining cluster 2
si_merged.loc[si_merged['Cluster Labels'] == 1, si_merged.columns[[1] + list(range(5, si_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,St. George,Sporting Goods Shop,Baseball Stadium,Sports Club,Breakfast Spot,Athletics & Sports
1,New Brighton,Dance Studio,Sports Club,Sporting Goods Shop,Breakfast Spot,Athletics & Sports
2,Stapleton,Breakfast Spot,Dance Studio,Sports Club,Sporting Goods Shop,Athletics & Sports
3,Rosebank,Breakfast Spot,Dance Studio,Sports Club,Sporting Goods Shop,Athletics & Sports
4,West Brighton,Breakfast Spot,Dance Studio,Sports Club,Sporting Goods Shop,Athletics & Sports
5,Grymes Hill,Dance Studio,Sports Club,Sporting Goods Shop,Breakfast Spot,Athletics & Sports
6,Todt Hill,Dance Studio,Sports Club,Sporting Goods Shop,Breakfast Spot,Athletics & Sports
8,Port Richmond,Dance Studio,Sports Club,Sporting Goods Shop,Breakfast Spot,Athletics & Sports
9,Mariner's Harbor,Dance Studio,Sports Club,Sporting Goods Shop,Breakfast Spot,Athletics & Sports
10,Port Ivory,Dance Studio,Sports Club,Sporting Goods Shop,Breakfast Spot,Athletics & Sports


In [65]:
#Examining cluster 3
si_merged.loc[si_merged['Cluster Labels'] == 2, si_merged.columns[[1] + list(range(5, si_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
7,South Beach,Athletics & Sports,Dance Studio,Sports Club,Sporting Goods Shop,Breakfast Spot


In [66]:
#Examining cluster 4
si_merged.loc[si_merged['Cluster Labels'] == 3, si_merged.columns[[1] + list(range(5, si_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
24,Park Hill,Athletics & Sports,Gym / Fitness Center,Dance Studio,Sports Club,Sporting Goods Shop


In [67]:
#Examining cluster 5
si_merged.loc[si_merged['Cluster Labels'] == 4, si_merged.columns[[1] + list(range(5, si_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
22,Silver Lake,Golf Course,Dance Studio,Sports Club,Sporting Goods Shop,Breakfast Spot


In [91]:
top

,Venue Category,Neighborhood
0,Baseball Stadium,St. George
1,Sporting Goods Shop,St. George
2,Breakfast Spot,"Pleasant Plains, West Brighton"
3,"Athletics & Sports, Gym / Fitness Center",Park Hill
4,Spa,"Lighthouse Hill, Richmond Town"
5,Gym,Sunnyside
6,Sports Club,Travis
7,"Sports Bar, Dance Studio",Annadale
8,"Sports Bar, Skating Rink",New Dorp Beach
9,Golf Course,Silver Lake


In [93]:
Final_Locations = ['Annadale', 'Lighthouse Hill','New Dorp Beach', 'Park Hill', 'Silver Lake']
#The above final locations have been derived from matching the 'top' dataframe elements with neighbors from each cluster.

In [101]:
si_merged

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Staten Island,St. George,40.644982,-74.079353,1,Sporting Goods Shop,Baseball Stadium,Sports Club,Breakfast Spot,Athletics & Sports
1,Staten Island,New Brighton,40.640615,-74.087017,1,Dance Studio,Sports Club,Sporting Goods Shop,Breakfast Spot,Athletics & Sports
2,Staten Island,Stapleton,40.626928,-74.077902,1,Breakfast Spot,Dance Studio,Sports Club,Sporting Goods Shop,Athletics & Sports
3,Staten Island,Rosebank,40.615305,-74.069805,1,Breakfast Spot,Dance Studio,Sports Club,Sporting Goods Shop,Athletics & Sports
4,Staten Island,West Brighton,40.631879,-74.107182,1,Breakfast Spot,Dance Studio,Sports Club,Sporting Goods Shop,Athletics & Sports
5,Staten Island,Grymes Hill,40.624185,-74.087248,1,Dance Studio,Sports Club,Sporting Goods Shop,Breakfast Spot,Athletics & Sports
6,Staten Island,Todt Hill,40.597069,-74.111329,1,Dance Studio,Sports Club,Sporting Goods Shop,Breakfast Spot,Athletics & Sports
7,Staten Island,South Beach,40.580247,-74.079553,2,Athletics & Sports,Dance Studio,Sports Club,Sporting Goods Shop,Breakfast Spot
8,Staten Island,Port Richmond,40.633669,-74.129434,1,Dance Studio,Sports Club,Sporting Goods Shop,Breakfast Spot,Athletics & Sports
9,Staten Island,Mariner's Harbor,40.632546,-74.150085,1,Dance Studio,Sports Club,Sporting Goods Shop,Breakfast Spot,Athletics & Sports


In [113]:
#lat_long = [[40.538114, -74.178549], [40.576506, -74.137927], [40.564255, -74.104327], [40.609190, -74.080157], [40.619193, -74.096290]]
lat = [40.538114, 40.576506, 40.564255, 40.609190, 40.619193]
lat1=pd.DataFrame(lat)
long = [-74.178549, -74.137927, -74.104327, -74.080157, -74.096290]
long1=pd.DataFrame(long)
lat1.columns =['latitude']
long1.columns =['longitude']
print(lat1)
print(long1)

    latitude
0  40.538114
1  40.576506
2  40.564255
3  40.609190
4  40.619193
   longitude
0 -74.178549
1 -74.137927
2 -74.104327
3 -74.080157
4 -74.096290


In [115]:
#Create the final map to be presented to the Corporates
map_final = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(lat1['latitude'], long1['longitude'], Final_Locations):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_final)  
    
map_final

# As a final note, i'd like to conclude by bringing to your notice that all the above data analysis has been based on an open source people's opinion platform (Foursquare) and it may or may not be accurate. 